In [ ]:
import csv
import os
import time
import random
import re

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def create_labeled_set():
    train_set = pd.DataFrame({'Files': 0, 'Labels': 0}, index=[0]).iloc[0:0]
    categories = os.listdir('debug_train')
    for category in categories:
        files = os.listdir('debug_train/' + category)
        labels = [category] * len(files)
        idx = [i for i in range(len(files))]
        temp_df = pd.DataFrame({'Files': files, 'Labels': labels}, index=idx)
        train_set = train_set.append(temp_df, ignore_index=True)
    train_set.to_csv('Train_List.csv')
    possible_labels = train_set['Labels'].unique()
    return possible_labels

In [ ]:
def partition_data(file_list, percent_validation):
    train_rows = 0
    valid_rows = 0
    counter = 0
    with open(file_list, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row)>0:
                if counter==0:
                    with open('Train_Partition.csv', 'w') as trainfile:
                        pass
                    with open('Valid_Partition.csv', 'w') as validfile:
                        pass
                    counter += 1
                else:
                    group = np.random.multinomial(1,[1.-percent_validation,
                                                     percent_validation])
                    counter += 1
                    
                    if np.argmax(group)==0:
                        with open('Train_Partition.csv', 'a',
                                  newline = None) as trainfile:
                            trainwriter = csv.writer(trainfile, delimiter=',')
                            trainwriter.writerow(row)
                        train_rows += 1
                    
                    if np.argmax(group)==1:
                        with open('Valid_Partition.csv', 'a',
                                  newline = None) as validfile:
                            validwriter = csv.writer(validfile, delimiter=',')
                            validwriter.writerow(row)  
    
    return train_rows, valid_rows

In [ ]:
def read_labeled_image_list(image_list_file):
    filenames = []
    labels = []
    counter = 0
    with open(image_list_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row)>0:
                if counter == 0:
                    counter += 1
                    pass
                else:
                    filename = "train/" + row[1]
                    filenames.append(filename)
                    label = row[2]
                    labels.append(int(label))
    
    return filenames, labels

In [ ]:
def read_file_format(input_queue, possible_labels):
    file_contents = tf.read_file(input_queue[0])

    example = tf.image.decode_jpeg(file_contents, channels=3)
    example = tf.random_crop(example, size=[500, 500, 3])
    example = tf.image.random_flip_left_right(example)
    example = tf.image.random_brightness(example, max_delta=0.25)
    example = tf.image.random_contrast(example, lower=0.75, upper=1.25)
    example = tf.cast(example, tf.float32)
    
    raw_label = input_queue[1]
    label = tf.one_hot(tf.where(tf.equal(possible_labels, raw_label))[0],
                       depth = possible_labels.shape[0], on_value = 1,
                       off_value = 0)
    label = label[0]
    return example, label

In [ ]:
def input_pipeline(file_list, unique_labels, batch_size = 3,
                   num_epochs = None, evaluation = False):   
    image_list, label_list = read_labeled_image_list(file_list)
    
    images = tf.convert_to_tensor(image_list, dtype=tf.string)
    labels = tf.convert_to_tensor(label_list, dtype=tf.int32)
    
    input_queue = tf.train.slice_input_producer([images, labels],
                                            num_epochs=num_epochs,
                                            shuffle=True)
    
    example, label = read_file_format(input_queue, unique_labels)

    min_after_dequeue = 10
    capacity = min_after_dequeue + 10
    example_batch, label_batch = tf.train.shuffle_batch(
      [example, label], batch_size=batch_size, capacity=capacity, 
        min_after_dequeue=min_after_dequeue)
    return example_batch, label_batch

In [ ]:
def variable_with_weight_decay(name, shape, stddev, wd):

    var = tf.Variable(name=name, initial_value=tf.truncated_normal(shape=shape,
                          stddev=stddev, dtype=tf.float32), dtype=tf.float32)
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

In [ ]:
def _activation_summary(x):
    tensor_name = x.name
    tensor_name = tensor_name.replace(':', '_')
    tensor_name = tensor_name.replace('(', '_')
    tensor_name = tensor_name.replace(')', '_')
    tensor_name = tensor_name.replace(' ', '_')

    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))


In [ ]:
def make_prediction(images):
    
    with tf.variable_scope('conv1') as scope:
        kernel = variable_with_weight_decay('weights',
                                             shape=[5, 5, 3, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = variable_with_weight_decay('biases',
                                             shape=[64],
                                             stddev=5e-2,
                                             wd=None)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1)
        
    
    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.dropout(pool1, keep_prob=0.5, name='norm1')
    
      # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = variable_with_weight_decay('weights',
                                             shape=[5, 5, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = variable_with_weight_decay('biases',
                                             shape=[64],
                                             stddev=5e-2,
                                             wd=None)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2)

    # norm2
    norm2 = tf.nn.dropout(conv2, keep_prob=0.5, name='norm2')

    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1], padding='SAME', name='pool2')
    
     # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        pool2_len = np.prod(pool2.get_shape().as_list()[1:])
        reshape = tf.reshape(pool2, [batch_size, pool2_len])
        weights = variable_with_weight_decay('weights', shape=[pool2_len, 384],
                                              stddev=0.04, wd=0.004)
        biases = variable_with_weight_decay('biases',
                                             shape=[384],
                                             stddev=5e-2,
                                             wd=None)
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases,
                            name=scope.name)
        _activation_summary(local3)

      # local4
    with tf.variable_scope('local4') as scope:
        weights = variable_with_weight_decay('weights', shape=[384, 192],
                                              stddev=0.04, wd=0.004)
        biases = variable_with_weight_decay('biases',
                                             shape=[192],
                                             stddev=5e-2,
                                             wd=None)
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases,
                            name=scope.name)
        _activation_summary(local4)

      # linear layer(WX + b),
      # We don't apply softmax here because
      # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
        # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = variable_with_weight_decay('weights', [192, len(unique_labels)],
                                              stddev=1/192.0, wd=0.0)
        biases = variable_with_weight_decay('biases',
                                            shape=[len(unique_labels)],
                                            stddev=5e-2,
                                            wd=None)
        softmax_linear = tf.add(tf.matmul(local4, weights), biases,
                                name=scope.name)
        _activation_summary(softmax_linear)
        
    return softmax_linear

In [ ]:
def calculate_loss(logits, labels):
    # Calculate the average cross entropy loss across the batch.
    labels = tf.cast(labels, tf.int64)
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
      labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [ ]:
def _add_loss_summaries(total_loss):
    # Compute the moving average of all individual losses and the total loss.
    loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
    losses = tf.get_collection('losses')
    loss_averages_op = loss_averages.apply(losses + [total_loss])

    # Attach a scalar summary to all individual losses and the total loss; do the
    # same for the averaged version of the losses.
    for l in losses + [total_loss]:
    # Name each loss as '(raw)' and name the moving average version of the loss
    # as the original loss name.
        l_name = l.name.replace(":", "_")
        tf.summary.scalar(l_name + '_raw_', l)
        tf.summary.scalar(l_name, loss_averages.average(l))

    return loss_averages_op

In [ ]:
def train(total_loss, global_step):
 
    # Variables that affect learning rate.
    num_batches_per_epoch = num_examples_per_train_epoch / batch_size
    decay_steps = int(num_batches_per_epoch * num_epochs_to_decay)

    # Decay the learning rate exponentially based on the number of steps.
    lr = tf.train.exponential_decay(
        initial_learning_rate, global_step,
        decay_steps, learning_rate_decay_factor, staircase=True)
    tf.summary.scalar('learning_rate', lr)

    # Generate moving averages of all losses and associated summaries.
    loss_averages_op = _add_loss_summaries(total_loss)

    # Compute gradients.
    with tf.control_dependencies([loss_averages_op]):
        opt = tf.train.AdagradOptimizer(lr)
        grads = opt.compute_gradients(total_loss)

    # Apply gradients.
    apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

    # Add histograms for trainable variables.
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)

    # Add histograms for gradients.
    for grad, var in grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)

    # Track the moving averages of all trainable variables.
    variable_averages = tf.train.ExponentialMovingAverage(
        moving_average_decay, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
        train_op = tf.no_op(name='train')

    return train_op

In [ ]:
def evaluate_accuracy(x, y_):
    y_hat = make_prediction(x)
    correct = tf.equal(tf.argmax(y_hat,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    tf.summary.scalar('validation_accuracy', accuracy)
    return accuracy

In [ ]:
unique_labels = create_labeled_set()
unique_labels = unique_labels.astype(int)

In [ ]:
file_list = 'Train_List.csv'

In [ ]:
train_size, valid_size = partition_data(file_list, percent_validation=0.2)

In [ ]:
train_partition = 'Train_Partition.csv'
valid_partition = 'Train_Partition.csv'

In [ ]:
global num_examples_per_train_epoch
num_examples_per_train_epoch = train_size

global num_epochs
num_epochs= 1

global batch_size
batch_size = 3

global moving_average_decay
moving_average_decay = 0.9999     # The decay to use for the moving average.

global num_epochs_to_decay
num_epochs_to_decay = 350.0    # Epochs after which learning rate decays.

global learning_rate_decay_factor
learning_rate_decay_factor = 0.1  # Learning rate decay factor.

global initial_learning_rate
initial_learning_rate = 0.00005       # Initial learning rate.

global momentum
momentum = 0.05

In [ ]:
logdir = 'TF_Logs'

In [ ]:
with tf.Graph().as_default():
    global_step = tf.Variable(0, trainable=False)
    
    train_example_feed, train_labels_feed = input_pipeline(
        train_partition, unique_labels=unique_labels,
        batch_size=batch_size, num_epochs=num_epochs)
    
    x = tf.placeholder(tf.float32, shape=[None, 500, 500, 3])
    y_ = tf.placeholder(tf.float32, shape = [None, 11])
    
    y_hat = make_prediction(x)
    
    loss = calculate_loss(y_hat, y_)
    
    train_op = train(loss, global_step=global_step)
    step = 0
    
    with tf.Session() as sess:
        # Start populating the filename queue.
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(logdir, sess.graph)

        init_op = tf.group(tf.global_variables_initializer(),
                           tf.local_variables_initializer())
        sess.run(init_op)
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        

        while not coord.should_stop():
            try:
                start_time = time.time()                
                
                example_batch, label_batch = sess.run([train_example_feed,
                                                       train_labels_feed])

                result, summary =  sess.run([train_op, merged],
                                            feed_dict={x: example_batch,
                                                       y_: label_batch})
                                
                
                
                writer.add_summary(summary, global_step.eval())

                step += 1
                print(step)
                duration = time.time() - start_time
                print(duration)

            except (tf.errors.OutOfRangeError,
                    tf.errors.InvalidArgumentError) as e:
                
                # When done, ask the threads to stop.
                coord.request_stop()
                
                valid_example_feed, valid_labels_feed = input_pipeline(
                    train_partition, unique_labels=unique_labels,
                    batch_size=valid_size, num_epochs=1
                )
                
                valid_examples, valid_labels = sess.run([valid_example_feed,
                                                       valid_labels_feed])
                
                sess.run(evaluate_accuracy, feed_dict={x: valid_examples,
                                                       y_: valid_labels})
                summary = sess.run(merged,
                                   feed_dict={
                                       x: valid_examples, y_: valid_labels
                                   })

                
        # Wait for threads to finish.
        coord.join(threads)
        sess.close()